<a href="https://colab.research.google.com/github/OJB-Quantum/Notebooks-for-Ideas/blob/main/MD_to_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Pandoc and LaTeX packages

!apt-get update -y
!apt-get install -y pandoc
!apt-get install -y texlive-latex-base texlive-latex-recommended texlive-latex-extra texlive-fonts-recommended texlive-xetex

# Remove the hash below if you want to use IBM Plex Sans font on the PDF output
# !apt-get install -y fonts-ibm-plex  # Install IBM Plex Sans font

The code below will prompt you to upload a markdown file so that it can be converted into a PDF. The downloadable PDF will open in a new tab. This version of the tool will be able to render LaTeX. Wait a couple seconds for the PDF to generate and open in a new tab.

In [ ]:
# 2. Create a custom LaTeX template    #

template_content = r'''
\documentclass{report}

\usepackage{graphicx}
\usepackage{longtable}
\usepackage{booktabs}    % Provides professional horizontal lines
\usepackage{array}
\usepackage{geometry}
\usepackage{lipsum}
\usepackage{tabularx}
\usepackage{multirow}
\usepackage{hyperref}
\usepackage[export]{adjustbox}
\usepackage{setspace}
\usepackage{ragged2e}    % For \justifying
% \usepackage{fontspec}    % For embedding  custom fonts like IBM Plex Sans, remove the '%' sign before '\usepackage'

% To set IBM Plex Sans as the main font, remove the '%' sign below
% \setmainfont{IBMPlexSans}

% Automatically scale images so they don't exceed margins:
\setkeys{Gin}{width=\linewidth, height=\textheight, keepaspectratio}

% Define \tightlist to avoid undefined control sequence error in Pandoc
\providecommand{\tightlist}{
  \setlength{\itemsep}{0pt}\setlength{\parskip}{0pt}
}

% --- TITLE BLOCK (centered) ---
\title{
\centering
\Huge\textbf{<insert desired document title>}\\
{\LARGE <insert desired organization name>}
}

\author{
\centering
\Large{<insert First Name Last Name>}
}

\date{\centering <insert desired date>}

% 1-inch margins all around
\geometry{left=1in, right=1in, top=1in, bottom=1in}

% 1.5 line spacing
\setstretch{1.5}

\begin{document}

% Title (centered)
\maketitle

% Justify all subsequent text:
\justifying

$body$

\end{document}
'''

with open('custom.latex', 'w') as f:
    f.write(template_content)


# 3. Prompt for file upload (Markdown) #

from google.colab import files
uploaded = files.upload()

import os
from IPython.display import display, HTML


# 4. Convert to PDF using the template #

for fn in uploaded.keys():
    base_name = os.path.splitext(fn)[0]
    pdf_filename = base_name + ".pdf"

    # Use Pandoc with XeLaTeX + custom template
    # Also parse pipe tables (+pipe_tables) and grid tables (+grid_tables)
    !pandoc "{fn}" \
            --from markdown+pipe_tables+grid_tables \
            --pdf-engine=xelatex \
            --template="custom.latex" \
            -o "{pdf_filename}"

    # Provide links to open/download the PDF
    if os.path.exists(pdf_filename):
        display(HTML(f'<a href="{pdf_filename}" target="_blank">Open PDF in new tab</a>'))
        files.download(pdf_filename)
    else:
        print(f"PDF generation failed for {fn}. Check the console output above for errors.")